<a href="https://colab.research.google.com/github/nicolashernandez/teaching_nlp/blob/main/static_representation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

L'objectif de ce notebook est de
- construire des représentations statiques clairsemées et denses
- observer leur utilisation sur une tâche de classification

Plusieurs modélisations de données seront explorées
1. term frequency model
2. TF-IDF
3. pretrained word2vec
4. trained word2vec

Dans le cadre de ce travail, vous ne travaillerez que sur un échantillon (sample) de données pour ne pas attendre trop longtemps la réalisation des traitements.


- jouer avec les prétraitements
- jouer avec les dimensions
- jouer avec les vectorisations
- obtener la meilleur performance

# A bit of theory

## Modèle "sac de mots" clairsemé

Le [**sac de mots** ou *BOW* pour *Bag Of Words* en anglais](https://fr.wikipedia.org/wiki/Sac_de_mots) est un modèle classique utilisé en Recherche d'Information (RI) pour représenter le contenu d'un document. Chaque document est décrit vis-à-vis d'un vocabulaire donné commun.

Différentes vues sont possibles :
- compter binairement si les mots du vocabulaire sont présents dans le document,
- compter les occurrences des mots du vocabulaire dans le document,
- pondérer les mots en tenant compte de leurs spécificités dans le document vis-à-vis des autres documents (on parle de pondération _tf-idf_).

La **vectorisation** est le processus qui désigne transformation des textes en vecteurs (de mots selon une modélisation *bow*).

On doit à [Karen Spärck Jones](https://fr.wikipedia.org/wiki/Karen_Sp%C3%A4rck_Jones) la proposition de la pondération _tf-idf_ des termes.
> « La spécificité d'un terme peut être quantifiée comme une fonction inverse du nombre de documents dans lesquels il apparaît. »

[Gérard Salton](https://fr.wikipedia.org/wiki/Gerard_Salton), quant à lui, est reconnu comme étant le père de la recherche d'information en ayant proposé une modélisation des documents dans un espace vectoriel.


* Karen Spärck Jones, « A statistical interpretation of term specificity and its application in retrieval », Journal of Documentation, vol. 28, no 1,‎ 1972, p. 11–21 (DOI 10.1108/eb026526)
* G. Salton , A. Wong , C. S. Yang, A vector space model for automatic indexing, Communications of the ACM, v.18 n.11, p. 613-620, novembre 1975

Par la suite nous utiliserons le module python sklearn qui offre des facilités pour pré-traiter (normaliser) et vectoriser aisément les textes.

On utilisera néanmoins un pré-traitement extérieur pour comparer différentes vectorisations concurrentes.

## Modèle dense statique

Les [**plongements de mots** (_word embeddings_ en anglais)](https://fr.wikipedia.org/wiki/Word_embedding) désignent le résultat de techniques récentes de vectorisation qui produisent des vecteurs denses de dimensions réduites, prédéfinies, et non corrélées avec la taille du vocabulaire (e.g. 100, 300, 500...).
Ces techniques reposent sur l'hypothèse distributionnelle de Harris qui veut que les mots apparaissant dans des contextes similaires ont des significations apparentées.

La méthode de référence est connue sous le nom [**Word2Vec** est attribuée à Mikolov et ses collègues (Google) en 2013](https://github.com/tmikolov/word2vec).
> Tomas Mikolov, Kai Chen, Greg Corrado et Jeffrey Dean, « Efficient Estimation of Word Representations in Vector Space », arXiv:1301.3781 [cs],‎ 16 janvier 2013

Les auteurs proposent deux architectures neuronales à 2 couches [CBOW (_continuous bag of words_) et SkipGram](https://fr.wikipedia.org/wiki/Word2vec#/media/Fichier:CBOW_eta_Skipgram.png).

> Le CBOW vise à prédire un mot étant donné son contexte. Skip-gram a une architecture symétrique visant à prédire les mots du contexte étant donné un mot en entrée. En pratique, le modèle CBOW est plus rapide à apprendre, mais le modèle skip-gram donne généralement de meilleurs résultats.

> La couche cachée contient quelques centraines de neurones et constitue à l'issue de l'entraînement le plongement représentant un mot. La couche de sortie permet d'implémenter une tâche de classification au moyen d'une softmax.

> L'apprentissage ne nécessite néanmoins aucun label, la vérité terrain étant directement déduite des données et plus particulièrement de la proximité des mots au sein du corpus d'entraînement. En ce sens, l'apprentissage de Word2vec constitue un apprentissage auto-supervisé.

Les vecteurs obtenus sont dits statiques (ou non contextuels) car ils restent tel quel quelle que soit l'occurrence du mot en contexte.

Un second modèle bien connu est celui de [**FastText** (Facebook)](https://github.com/facebookresearch/fastText) qui propose de traiter la variabilité morphologique des mots en construisant des vecteurs non pas pour des mots mais pour des sous-mots (séquence de caractères). Le lecteur d'un mot est la somme de tous les vecteurs des sous-mots le composant.

Cette approche est indépendante de la langue, et montre de meilleurs résultats que word2vec sur des tâches syntaxiques, surtout quand le corpus d'entraînement est petit. Word2vec est légèrement meilleur pour des tâches sémantiques. Un des avantage de FastText est de pouvoir fournir des vecteurs mêmes pour les mots hors vocabulaires.

Plusieurs **librairies permettent de créer, charger, sauver et manipuler des modèles de plongements de mots**. Nous allons utiliser _gensim_ qui permet aussi bien de travailler avec des modèles [word2vec](https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html#sphx-glr-auto-examples-tutorials-run-word2vec-py) que [fasttext](https://radimrehurek.com/gensim/auto_examples/tutorials/run_fasttext.html#sphx-glr-auto-examples-tutorials-run-fasttext-py).  

Ici une [Comparison of FastText and Word2Vec](https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/Word2Vec_FastText_Comparison.ipynb)



# Environment

In [1]:
from tqdm import tqdm
import pandas as pd
pd.options.display.max_colwidth = 200
import numpy as np
import re
import json

# Dataset

Prefer to load preprocessed data, it would be faster.

## Load and preprocess the IMDB dataset

For our experiment, we use the [Large Movie Review Dataset](https://huggingface.co/datasets/stanfordnlp/imdb) from [Learning Word Vectors for Sentiment Analysis](https://aclanthology.org/P11-1015) (Maas et al., ACL 2011).

The dataset was developed for binary sentiment classification task.

It contains
* a set of 25,000 highly polar movie reviews for training,
* 25,000 for testing
* and additional 50,000 unlabeled data for use as well.




In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset

imdb_dataset = load_dataset("stanfordnlp/imdb")
imdb_dataset

ModuleNotFoundError: No module named 'datasets'

In [ ]:
raw_train_df = imdb_dataset['train'].to_pandas()
raw_test_df = imdb_dataset['test'].to_pandas()
raw_unsupervised_df = imdb_dataset['unsupervised'].to_pandas()

raw_train_df

,text,label
0,I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if i...,0
1,"""I Am Curious: Yellow"" is a risible and pretentious steaming pile. It doesn't matter what one's political views are because this film can hardly be taken seriously on any level. As for the claim t...",0
2,If only to avoid making this type of film in the future. This film is interesting as an experiment but tells no cogent story.<br /><br />One might feel virtuous for sitting thru it because it touc...,0
3,"This film was probably inspired by Godard's Masculin, féminin and I urge you to see that film instead.<br /><br />The film has two strong elements and those are, (1) the realistic acting (2) the i...",0
4,"Oh, brother...after hearing about this ridiculous film for umpteen years all I can think of is that old Peggy Lee song..<br /><br />""Is that all there is??"" ...I was just an early teen when this s...",0
...,...,...
24995,"A hit at the time but now better categorised as an Australian cult film. The humour is broad, unsubtle and, in the final scene where a BBC studio fire is extinguished by urinating on it, crude. Co...",1
24996,"I love this movie like no other. Another time I will try to explain its virtues to the uninitiated, but for the moment let me quote a few of pieces the remarkable dialogue, which, please remember,...",1
24997,"This film and it's sequel Barry Mckenzie holds his own, are the two greatest comedies to ever be produced. A great story a young Aussie bloke travels to england to claim his inheritance and meets ...",1
24998,"'The Adventures Of Barry McKenzie' started life as a satirical comic strip in 'Private Eye', written by Barry Humphries and based on an idea by Peter Cook. McKenzie ( 'Bazza' to his friends ) is a...",1


define data cleaning method

In [ ]:
import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords

# optional
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

non_English_character_compiled_pattern = re.compile('[^a-zA-Z]')

def preprocess (raw_text):
  # remove non English character
  preprocessed_text = re.sub(non_English_character_compiled_pattern,' ',raw_text)
  # lower casing
  preprocessed_text = preprocessed_text.lower()
  # split as a list of words
  preprocessed_text = preprocessed_text.split()
  # remove stopwords
  preprocessed_text = [word for word in preprocessed_text if not word in stopwords.words('english')]
  # stemming
  # preprocessed_text = [ps.stem(word) for word in preprocessed_text]
  # join words again to form the text
  preprocessed_text = ' '.join(preprocessed_text)
  return preprocessed_text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


preprocess training data

```
100%|██████████| 25000/25000 [12:31<00:00, 33.27it/s]
```

In [ ]:
# preprocess X
X_train = []
for raw_text in tqdm(raw_train_df['text']):
  X_train.append(preprocess(raw_text))
X_train_np = np.array(X_train)

# Y
Y_train = list(raw_train_df['label'])
Y_train_np = np.array(Y_train)


# to backup and visualize
train_df = pd.DataFrame({'text': X_train, 'label': Y_train})
train_df.to_csv('imdb_train.csv', index=False)
train_df

preprocess test data

```
100%|██████████| 25000/25000 [14:27<00:00, 28.83it/s]
```



In [ ]:
# preprocess X
X_test = []
for raw_text in tqdm(raw_test_df['text']):
  X_test.append(preprocess(raw_text))
X_test_np = np.array(X_test)

# Y
Y_test = list(raw_test_df['label'])
Y_test_np = np.array(Y_test)

# to backup and visualize
test_df = pd.DataFrame({'text': X_test, 'label': Y_test})
test_df.to_csv('imdb_test.csv', index=False)
test_df

100%|██████████| 25000/25000 [14:27<00:00, 28.83it/s]


preprocess unsupervised data


```
100%|██████████| 50000/50000 [23:22<00:00, 35.64it/s]
```

In [ ]:
imdb_dataset['unsupervised']['label'][:10]

[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]

In [ ]:
#
X_unsupervised = []
for raw_text in tqdm(raw_unsupervised_df['text']):
  X_unsupervised.append(preprocess(raw_text))

#
X_unsupervised_np = np.array(X_unsupervised)

#
Y_unsupervised = list(raw_unsupervised_df['label'])

#
test_df = pd.DataFrame({'text': X_unsupervised, 'label': Y_unsupervised})
test_df.to_csv('imdb_unsupervised.csv', index=False)

## Load preprocessed dataset

Import manually the files in the local file sytem, then run the following code.

The data has been preprocessed with the `preprocess` method without stemming.

In [4]:
#
train_df = pd.read_csv('imdb_train.csv')
train_df = train_df.dropna(axis=0)
train_df['label'] = train_df['label'].astype(int)
X_train = list(train_df['text'])
Y_train = list(train_df['label'])

#
test_df = pd.read_csv('imdb_test.csv')
#print(test_df.shape)
#print(test_df.isnull().sum())
test_df = test_df.dropna(axis=0)
test_df['label'] = test_df['label'].astype(int)
#print(test_df.shape)
X_test = list(test_df['text'])
Y_test = list(test_df['label'])

#
X_train_np = np.array(X_train)
Y_train_np = np.array(Y_train)
X_test_np = np.array(X_test)
Y_test_np = np.array(Y_test)
#X_unsupervised_np = np.array(X_unsupervised)

#
unsupervised_df = pd.read_csv('imdb_unsupervised.csv')
X_unsupervised = list(unsupervised_df['text'])
#Y_unsupervised = list(unsupervised_df['label'])

#
train_df

,text,label
0,rented curious yellow video store controversy surrounded first released also heard first seized u customs ever tried enter country therefore fan films considered controversial really see br br plo...,0
1,curious yellow risible pretentious steaming pile matter one political views film hardly taken seriously level claim frontal male nudity automatic nc true seen r rated films male nudity granted off...,0
2,avoid making type film future film interesting experiment tells cogent story br br one might feel virtuous sitting thru touches many important issues without discernable motive viewer comes away n...,0
3,film probably inspired godard masculin f minin urge see film instead br br film two strong elements realistic acting impressive undeservedly good photo apart strikes endless stream silliness lena ...,0
4,oh brother hearing ridiculous film umpteen years think old peggy lee song br br early teen smoked fish hit u young get theater although manage sneak goodbye columbus screening local film museum be...,0
...,...,...
24995,hit time better categorised australian cult film humour broad unsubtle final scene bbc studio fire extinguished urinating crude contains every cliche traditional australian pilgrimage old country ...,1
24996,love movie like another time try explain virtues uninitiated moment let quote pieces remarkable dialogue please remember tongue cheek aussies poms understand everyone else well br br title song ly...,1
24997,film sequel barry mckenzie holds two greatest comedies ever produced great story young aussie bloke travels england claim inheritance meets mates loveable innocent br br chock block full great say...,1
24998,adventures barry mckenzie started life satirical comic strip private eye written barry humphries based idea peter cook mckenzie bazza friends lanky loud hat wearing australian whose two main inter...,1


## Sample

In order to make the process faster, work on a sample

In [17]:
# select a sample then split it into training and testing sets
from sklearn.utils import resample
from sklearn.model_selection import train_test_split

sample_data = resample(train_df, n_samples = 1000, random_state = 42)
print (sample_data.shape)

X_train, X_test, Y_train, Y_test = train_test_split(sample_data['text'], sample_data['label'], test_size=0.2, random_state=45 )
X_train_np = np.array(X_train)
Y_train_np = np.array(Y_train)
X_test_np = np.array(X_test)
Y_test_np = np.array(Y_test)

# FURTHER  perform your own preprocess on the raw dataset


(1000, 2)


In [19]:
from collections import Counter
print ('Y_train distribution', list(Counter(Y_train).items()))
print ('Y_test distribution', list(Counter(Y_test).items()))

Y_train distribution [(0, 388), (1, 412)]
Y_test distribution [(1, 92), (0, 108)]


# Approach 1: term frequency model

Le code suivant réalise une vectorisation du corpus en comptant les occurrences des mots. La vectorisation prend en charge la construction d'un vocabulaire sur le corpus ainsi que quelques pré-traitements de normalisation linguistiques. Enfin le résultat de la vectorisation est affiché sous la forme d'une matrice **document-terme** (les documents sont en ligne et les mots en colonne).
Nous aurons une matrice de dimension _nb de documents * taille du vocabulaire_.

Here we limit the number of dimensions to ```max_features=1000```.

Do not hesite to have a look at all the parameters of [CountVectorizer](https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html).

In [20]:
from sklearn.feature_extraction.text import CountVectorizer

# define the vectorizer
count_vectorizer = CountVectorizer(stop_words= None, max_features=1000) #stop_words='english')

# display the configuration of the vectorizer
print (count_vectorizer)

# Learn the vocabulary (fit) and transform documents to document-term matrix (i.e. perform the vectorization)
X_train_counter_matrix = count_vectorizer.fit_transform(X_train_np)
print ('Matrix dimensions:', X_train_counter_matrix.get_shape())
count_vectorizer.transform

# Transform documents to document-term matrix (i.e. perform the vectorization) by using Vocabulary learned
X_test_counter_matrix = count_vectorizer.transform(X_test_np)
print ('Matrix dimensions:', X_test_counter_matrix.get_shape())

# get all unique words in the corpus (the vocabulary and also the names of the matrix columns/features)
vocab = count_vectorizer.get_feature_names_out() # .get_feature_names() # for sklearn_version >= 1.0
print ('Vocabulary size:', len(vocab))

# show document-term matrix
X_train_counter_matrix_array = X_train_counter_matrix.toarray()
pd.DataFrame(X_train_counter_matrix_array, columns=vocab)

CountVectorizer(max_features=1000)
Matrix dimensions: (800, 1000)
Matrix dimensions: (200, 1000)
Vocabulary size: 1000


,ability,able,absolutely,accent,across,act,acting,action,actor,actors,...,writing,written,wrong,year,years,yes,yet,york,young,zombie
0,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,2,0,2,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
796,0,0,1,0,2,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
797,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
798,0,0,0,0,0,0,0,4,0,0,...,0,0,0,0,0,0,0,0,0,0


train

In [21]:
from sklearn.naive_bayes import MultinomialNB
MNB_counter_model = MultinomialNB().fit(X_train_counter_matrix, Y_train_np)

predict

In [22]:
Y_test_prediction = MNB_counter_model.predict(X_test_counter_matrix)

evaluate

In [23]:
from sklearn.metrics import accuracy_score, classification_report
print(accuracy_score(Y_test,Y_test_prediction))
print(classification_report(Y_test,Y_test_prediction))

0.79
              precision    recall  f1-score   support

           0       0.81      0.80      0.80       108
           1       0.77      0.78      0.77        92

    accuracy                           0.79       200
   macro avg       0.79      0.79      0.79       200
weighted avg       0.79      0.79      0.79       200



# Approach 2: TF-IDF model

La vectorisation avec occurrences présente des limites lorsqu'elle est utilisée sur de large corpus. En effet, le modèle présume que l'importance des mots est fonction de sa fréquence et qu'un mot plus fréquent qu'un autre dans un document est plus discriminant que l'autre. Le problème intervient quand un mot fréquent, supposé important, apparaît dans plusieurs documents. Le fait qu'il apparaisse dans plusieurs documents peut au final le rendre moins discriminant que d'autres pourtant moins fréquents. Le problème vient du fait que l'on prenne des valeurs absolues.

Le modèle **TF-IDF** vise à solutionner ce problème en normalisant le compte des occurrences. TF-IDF correspond à _Term Frequency-Inverse Document Frequency_.

On définit le TF-IDF comme suit: `tfidf = tf x idf`
* `tfidf(w, D)` est le score TF-IDF du mot `w` dans le document `D`
* `tf(w, D)` représente le nombre d'occurrence du terme `w` dans le document `D`
* `idf(w, D)` représente la fréquence inverse documentaire du terme `w`, qui peut être calculée comme le log du nombre total de documents dans le corpus `C` divisé par la fréquence documentaire du terme `w` (i.e. le nombre de documents du corpus `C` dans lequel le terme `w` se produit).


Here the definition of the [TfidfVectorizer](https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html).

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

# define the vectorizer
tfidf_vectorizer = # TODO

# Learn the vocabulary (fit) and transform documents to document-term matrix (i.e. perform the vectorization)
X_train_tfidf_matrix = # TODO
X_test_tfidf_matrix = # TODO

# train
# TODO

# predict
# TODO

# evaluate
# TODO

0.81
              precision    recall  f1-score   support

           0       0.86      0.78      0.82       108
           1       0.76      0.85      0.80        92

    accuracy                           0.81       200
   macro avg       0.81      0.81      0.81       200
weighted avg       0.81      0.81      0.81       200



# Approach 3: existing static dense models


## Load existing pretrain models

Le dépôt [gensim-data](https://github.com/RaRe-Technologies/gensim-data) contient quelques corpus et modèles pré-entraînés librement disponibles.

Jean-Philippe Fauconnier met des [modèles word2vec à disposition pour le français ](https://fauconnier.github.io/#data).

In [25]:
import gensim.downloader as gensim_api

# show info about available models/datasets
print(json.dumps(gensim_api.info(), indent=4))

# load a model
#w2v_model = gensim_api.load("glove-twitter-25")
w2v_model = gensim_api.load("glove-twitter-50")
#w2v_model = gensim_api.load("glove-twitter-100")
#w2v_model = gensim_api.load("glove-wiki-gigaword-100")
#w2v_model = gensim_api.load('word2vec-google-news-300')

{
    "corpora": {
        "semeval-2016-2017-task3-subtaskBC": {
            "num_records": -1,
            "record_format": "dict",
            "file_size": 6344358,
            "reader_code": "https://github.com/RaRe-Technologies/gensim-data/releases/download/semeval-2016-2017-task3-subtaskB-eng/__init__.py",
            "license": "All files released for the task are free for general research use",
            "fields": {
                "2016-train": [
                    "..."
                ],
                "2016-dev": [
                    "..."
                ],
                "2017-test": [
                    "..."
                ],
                "2016-test": [
                    "..."
                ]
            },
            "description": "SemEval 2016 / 2017 Task 3 Subtask B and C datasets contain train+development (317 original questions, 3,169 related questions, and 31,690 comments), and test datasets in English. The description of the tasks and the collect

## Obtenir les mots similaires

Pour chaque question ci-dessous, jouez le jeu et prenez le temps faire des propositions de réponses avant d'exécuter le code qui permettra de consulter la connaissance du modèle et connaître ce qu'il répondrait.

Si je vous dis 'roi', vous pensez à quoi ? Faire quelques propositions de synonymes ou de mots substituables sémantiquement proches. La méthode `most_similar` affichera les 10 mots les plus proches d'un mot donné, du plus similaire au moins similaire, avec pour chacun un score de similarité avec le mot donné (scores décroissants donc).

In [ ]:
# obtenir les mots similaires à 1 mot
w2v_model.most_similar("king")

Si je vous demande de me donner des mots relatifs à des 'palais' et 'paris', à quoi pensez-vous ? Pour information, la méthode accepte une liste de mots en paramètres.

In [ ]:
# obtenir les mots similaires relatifs à une liste
w2v_model.most_similar(['berlin', 'paris'])

Si j'ajoute les vecteurs de roi et de femme et que je retire le vecteur homme qu'est ce que j'obtiens ? Répondez avant d'exécuter le code ci-dessous.



In [ ]:
# Si j'ajoute les vecteurs de roi et de femme et que je retire le vecteur homme qu'est ce que j'obtiens ?
w2v_model.most_similar(positive = ['king', 'woman'], negative = ['man'])


Même question mais si j'ajoute les vecteurs de 'paris' et de 'japon' et que je retire le vecteur de 'france'.

Trouver des exemples qui montrent les connaissances de genre, de nombre du modèles
* tree + grape - apple =



In [ ]:
# TODO

Trouver des exemples d'opérations (e.g. `computer programmer - man + worman =  housekeeper ?`) et de mots similaires  qui montrent des biais dans le modèles (e.g. quels sont les mots similaires de `gay` de `awful`?).

TODO


## Visualiser les plongements lexicaux dans un graph en 2D

Pour ce faire, il faut transformer les vecteurs de n dimensions à des vecteurs à 2 dimensions. La réduction des dimensions est effectuée à l'aide d'une [analyse en composantes principales (ACP ou PCA pour _Principal Component Analysis_ en anglais)](https://fr.wikipedia.org/wiki/Analyse_en_composantes_principales).

In [ ]:
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt
#plt.style.use('ggplot')
#words = sum([[k] + v for k, v in similar_words.items()], [])
#words = ['roi', 'reine']
# soit une liste de mots à projeter
words = ['palais', 'église', 'cathédrale', 'monastère', 'route', 'train', 'bateau', 'calèche', 'voiture', 'armée', 'soldat', 'bataille']
wvs = model[words]

# Application de la transformation PCA qui réduit les vecteurs à 2 dimensions
pca = PCA(n_components=2)
#np.set_printoptions(suppress=True)
P = pca.fit_transform(wvs)
labels = words

# Affichage
fig = plt.figure(figsize=(10, 8))
plt.scatter(P[:, 0], P[:, 1], c='lightgreen', edgecolors='g')
for label, x, y in zip(labels, P[:, 0], P[:, 1]):
    plt.annotate(label, xy=(x*1.05, y*1.05), xytext=(0, 0), textcoords='offset points')

* Est-ce que les synonymes se retrouvent bien dans les mêmes zones spatiales ? Vous pouvez tester avec d'autres listes de mots. TODO

## Visualiser les plongements lexicaux en 3D dynamique à l'aide du _projector de tensorflow_

Sur https://radimrehurek.com/gensim/scripts/word2vec2tensor.html, on peut lire comment convertir un modèle w2v (éventuellement produit par gensim) en modèle tsv, puis comment le visualiser avec le projector de tensorflow :

1. Convert your word-vector with word2vec2tensor method ou le script gensim.scripts.word2vec2tensor
2. Open http://projector.tensorflow.org/
3. Click “Load Data” button from the left menu.
4. Select “Choose file” in “Load a TSV file of vectors.” and choose “/tmp/my_model_prefix_tensor.tsv” file.
5.  Select “Choose file” in “Load a TSV file of metadata.” and choose “/tmp/my_model_prefix_metadata.tsv” file.
6. ???
7. PROFIT!

Le code ci-dessous définit des fonctions de conversion au format de tensorflow soit depuis le format gensim-w2v soit le format w2v binaire natif.

In [ ]:
import gensim
from gensim.scripts.word2vec2tensor import word2vec2tensor

def convert_gensim_w2v_to_w2v (gensim_w2v_in_path, w2v_out_path):
  """
  convert a model from gensim_w2v format to w2v (orginal) format
  """
  w2v_model = KeyedVectors.load(gensim_w2v_in_path)
  vectors = w2v_model.wv
  # save memory
  # del model

  # The trained word vectors can also be stored/loaded from a format compatible
  # with the original word2vec implementation via Word2Vec.wv.save_word2vec_format
  # and gensim.models.keyedvectors.KeyedVectors.load_word2vec_format().
  vectors.save_word2vec_format(w2v_out_path, binary = True)

def convert_w2v_to_tsv (w2v_in_path, tsv_out_path):
  """
  convert a model from w2v original format to tsv format
  """
  # When running word2vec2tensor with a file resulting from
  # save_word2vec_format, we obtain the following error:
  # UnicodeDecodeError: 'utf-8' codec can't decode byte 0xbd in position 0: invalid start byte
  # To solve the issue, I have to load with load_word2vec_format the saved file
  # and save it again with save_word2vec_format
  w2v_model = gensim.models.KeyedVectors.load_word2vec_format(w2v_in_path, binary=True, unicode_errors='ignore')
  w2v_model.wv.save_word2vec_format(w2v_in_path+".tmp", binary = True)
  word2vec2tensor(w2v_in_path+".tmp", tsv_out_path,  binary = True)

def convert_gensim_w2v_to_tsv (gensim_w2v_in_path, tsv_out_path):
  """
  convert a model from gensim w2v format to tsv format
  """
  convert_gensim_w2v_to_w2v (gensim_w2v_in_path, gensim_w2v_in_path+".tmp")
  convert_w2v_to_tsv (gensim_w2v_in_path+".tmp", tsv_out_path)

Convertissons le modèle public récupéré

In [ ]:
convert_w2v_to_tsv(w2v_model_path, w2v_model_path+".tsv")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).



Télécharger les 3? fichiers produits et chargez les dans projector tensorflow. Si c'est trop compliqué, le projector vient avec des modèles préchargés.

* Observez-vous des zones plus denses que d'autres ? Qu'est ce que cela peut vouloir signifier ?
* Testez les labels 3D, de cliquer sur un point/mot (fixer le voisinage à la valeur minimale) pour observer l'illumination d'une zone, chercher un mot, visualiser 'isolate 6 points'.
* Testez aussi un des tensors found disponible comme Word2Vec 10K ou all.

TODO

## Comparer et évaluer deux modèles

[`gensim` implémente la comparaison de modèles](https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/Word2Vec_FastText_Comparison.ipynb) selon la tâche de **raisonnement analogique** telle que décrite à la [section 4.1 du papier de 2013 de Mikolov et al.](https://arxiv.org/pdf/1301.3781v3.pdf).

```
:capital-common-countries
Athens Greece Baghdad Iraq
Athens Greece Bangkok Thailand
...
:capital-world
Algiers Algeria Baghdad Iraq
Ankara Turkey Dublin Ireland
...
: city-in-state
Chicago Illinois Houston Texas
Chicago Illinois Philadelphia Pennsylvania
...
: gram1-adjective-to-adverb
amazing amazingly apparent apparently
amazing amazingly calm calmly
...
```

D'autres évaluations intrinsèques sont possibles comme le [calcul d'un coefficient de corrélation entre un taux de similarité calculée sur la base d'une appréciation humaine et un score de similarité cosinus entre des représentations Word2Vec](https://nlp-ensae.github.io/materials/course2/).



Ci-dessous nous mettons en oeuvre la tâche de raisonnement analogique de Mikolov et al.

In [ ]:
# download the file questions-words.txt to be used for comparing word embeddings
!wget https://raw.githubusercontent.com/tmikolov/word2vec/master/questions-words.txt

--2024-11-04 07:45:22--  https://raw.githubusercontent.com/tmikolov/word2vec/master/questions-words.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 603955 (590K) [text/plain]
Saving to: ‘questions-words.txt’

questions-words.txt 100%[===================>] 589.80K  --.-KB/s    in 0.1s    

2024-11-04 07:45:23 (3.85 MB/s) - ‘questions-words.txt’ saved [603955/603955]



In [ ]:
# un oeil sur les n premières lignes du fichier
!head questions-words.txt

Définition de la méthode de calcul de la performance de resolution de la tâche d'analogie

In [ ]:
def print_accuracy(model, questions_file):
    print('Evaluating...\n')
    acc = model.accuracy(questions_file)
    #acc = model.wv.evaluate_word_analogies(questions_file)

    sem_correct = sum((len(acc[i]['correct']) for i in range(5)))
    sem_total = sum((len(acc[i]['correct']) + len(acc[i]['incorrect'])) for i in range(5))
    sem_acc = 100*float(sem_correct)/sem_total
    print('\nSemantic: {:d}/{:d}, Accuracy: {:.2f}%'.format(sem_correct, sem_total, sem_acc))

    syn_correct = sum((len(acc[i]['correct']) for i in range(5, len(acc)-1)))
    syn_total = sum((len(acc[i]['correct']) + len(acc[i]['incorrect'])) for i in range(5,len(acc)-1))
    syn_acc = 100*float(syn_correct)/syn_total
    print('Morphologic: {:d}/{:d}, Accuracy: {:.2f}%\n'.format(syn_correct, syn_total, syn_acc))
    return (sem_acc, syn_acc)

Exécution de l'évaluation

In [ ]:
#
word_analogies_file = 'questions-words.txt'

print('\nLoading Word2Vec embeddings')
w2v_model = KeyedVectors.load(w2v_model_path)
print('Accuracy for Word2Vec:')
print_accuracy(w2v_model, word_analogies_file)

print('\nLoading FastText embeddings')
ft_model = KeyedVectors.load(ft_model_path)
print('Accuracy for FastText (with n-grams):')
print_accuracy(ft_model, word_analogies_file)


TODO

* Lequel des deux modèles donnent les meilleurs résultats sur l'analyse morphologique ? Sur l'analyse sémantique ? Est-ce cohérent de ce que vous connaissez des modèles ?
* Relancez la construction des modèles puis leur comparaison. Obtenez-vous les mêmes scores de performance ? Pourquoi ?
* Les données d'entraînement sont des romans classiques issus de la collection Gutenberg. Si les données avaient été issues de la Wikipedia, quels résultats auraient pu changer ? Si vous souhaitez tester, ci-dessous je vous donne un snippet de code qui récupère une version normalisée de la wikipédia et qui lance la contruction des modèles w2v et ft. Cela prendra qqs minutes...
* Selon vous, dans une perspective de comparaison de modèles, est-ce important de construire ceux-ci sur les mêmes données ?

In [ ]:
# WARNING: ce qui suit est optionnel !

# récupération d'un corpus normalisé de la wikipédia-en
# une string tokenisée avec le caractère espace de mots pleins
!mkdir data
!wget -nc http://mattmahoney.net/dc/text8.zip -P data
!unzip data/text8.zip -d data

text8_path = 'data/text8'

# Text8Corpus class for reading space-separated words file
from gensim.models.word2vec import Text8Corpus

# Construction des modèles w2v et ft avec text8
%time w2v_model = Word2Vec(Text8Corpus(text8_path), **params)
%time ft_model = FastText(Text8Corpus(text8_path), **params)